# Analyzing My Room Condition


## Import Modules


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
from datetime import datetime
import csv
import os

## Global Variables


In [ ]:
dir_master = "../data/"  # master path
font_path = "NanumBarunGothic-YetHangul.otf"

file_list: list = []
userInput = None
FILE_PATH: str = None
data_list: list = []
df: pd.DataFrame

CURRTIME = str(datetime.now())[2:19].replace("-", "").replace(":", "").replace(" ", "-")
print(CURRTIME)

## Select Data


In [ ]:
# 모든 데이터 목록 가져오기
def GetDataList():
    global file_list
    file_list = os.listdir(dir_master)  # 파일 목록 가져오기
    file_list.sort()  # 정렬

    non_txt_files = []  # 텍스트 파일 아닌 폴더, 파일 등 저장

    # 텍스트 파일 아닌 폴더, 파일 등 추출
    for item in file_list:
        if not item.endswith(".txt"):
            non_txt_files.append(item)

    # .txt 파일 아닌 것들 모두 삭제
    for item in non_txt_files:
        file_list.pop(file_list.index(item))

    # 파일 목록 출력
    for i in range(len(file_list)):
        print(i, end=": ")
        print(file_list[i])
    print(f"Number Of Files >> {len(file_list)}")


# 처리할 데이터 선택하고 경로 저장
def SelectData():
    global FILE_PATH, userInput
    userInput = int(input(f"Select File Index{file_list}"))
    print(f"UserInput >> {userInput}", end=", ")
    print(f"Selected >> {file_list[userInput]}")

    FILE_PATH = f"{dir_master}{file_list[userInput]}"
    print(f"\nFile Path Set as >> {FILE_PATH}")

In [ ]:
def FileOpenRead():
    print(f"Opening {FILE_PATH}")
    global data_list

    # 파일 읽기
    with open(FILE_PATH, "r") as f:
        data = f.readlines()
    FILE = data
    # print(FILE)

    data_list = []
    for item in FILE:
        item = item.replace("\n", "")
        item = item.split()
        temp = f"{item[0]} {item[1]}"
        item.insert(0, temp)
        temp = [">>", item[1], item[2]]
        for i in temp:
            index = item.index(i)
            item.pop(index)
        data_list.append(item)
    # print(data_list)

    # print(data_list[0])
    MAX = len(data_list)
    print(f"Max Data Length >> {MAX}")

## Convert to DataFrame


In [ ]:
def CreateDataFrame():
    global df
    df = pd.DataFrame(data_list)
    df.rename(
        columns={
            0: "Date",
            # 1: "Time",
            1: "Temperature",
            2: "Humidity",
            3: "Voltage",
            4: "DustDensity",
        },
        inplace=True,
    )
    df = df.astype(
        {
            "Temperature": "int",
            "Humidity": "float",
            "Voltage": "float",
            "DustDensity": "float",
        }
    )
    df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y %H:%M:%S")
    df.set_index("Date", inplace=True)
    # print(df.info())
    # print(df.head())
    # print(df.dtypes)

## Check Data


### null check


In [ ]:
def dfNullCheck():
    global df
    print(df.isnull().sum())

    # ###################

    # df = df.fillna(method="pad", axis=0)
    # # df = df.fillna(df.interpolate(), axis=0)

    # # print(df)
    # df.to_csv("MYROOM.csv")
    # ###################

    # print(df.isnull().sum())

### Data Shift


In [ ]:
# temp_dust = df["DustDensity"]
# print(temp_dust)
# df["DustDensity"] = df["DustDensity"].shift(10)
# print(df["DustDensity"])

# print(df)
# df.shift(10, axis=1)
# print(df)

### Remove wrong data


In [ ]:
def DisplayColSum_old():
    print("Temperature", end=": ")
    print(df["Temperature"].min(), end=" ")
    print(df["Temperature"].mean(), end=" ")
    print(df["Temperature"].median(), end=" ")
    print(df["Temperature"].max())

    print("Humidity", end=": ")
    print(df["Humidity"].min(), end=" ")
    print(df["Humidity"].mean(), end=" ")
    print(df["Humidity"].median(), end=" ")
    print(df["Humidity"].max())

    print("Voltage", end=": ")
    print(df["Voltage"].min(), end=" ")
    print(df["Voltage"].mean(), end=" ")
    print(df["Voltage"].median(), end=" ")
    print(df["Voltage"].max())

    print("DustDensity", end=": ")
    print(df["DustDensity"].min(), end=" ")
    print(df["DustDensity"].mean(), end=" ")
    print(df["DustDensity"].median(), end=" ")
    print(df["DustDensity"].max())


def DisplayColSum():
    data_summary = {
        "Temperature": [
            df["Temperature"].min(),
            df["Temperature"].mean(),
            df["Temperature"].median(),
            df["Temperature"].max(),
        ],
        "Humidity": [
            df["Humidity"].min(),
            df["Humidity"].mean(),
            df["Humidity"].median(),
            df["Humidity"].max(),
        ],
        "Voltage": [
            df["Voltage"].min(),
            df["Voltage"].mean(),
            df["Voltage"].median(),
            df["Voltage"].max(),
        ],
        "DustDensity": [
            df["DustDensity"].min(),
            df["DustDensity"].mean(),
            df["DustDensity"].median(),
            df["DustDensity"].max(),
        ],
    }
    row_names = ["min", "mean", "median", "max"]
    df_sum = pd.DataFrame(data_summary, index=row_names)
    print(df_sum)


def CheckIssue(column: str):
    count = 0
    global df
    for item in df[column]:
        if abs(df[column].median() - item) > 80:
            delete_row = df[df[column] == item].index
            df = df.drop(delete_row)
            count += 1
            # print(df[column])
    print(f"Dropped {count} columns at {column}")
    return count

## Draw Graph


In [ ]:
def DrawGraph_old(types: str):
    # Adjust max/min value
    TDelta = 1
    HDelta = 1
    VDelta = 0.3
    DDelta = 3

    # Y Data
    y1 = df["Temperature"]
    y2 = df["Humidity"]
    y1_name = "Temperature"
    y2_name = "Humidity"
    # Y axis Max & Min Value
    y1_max = df["Temperature"].max() + TDelta
    y1_min = df["Temperature"].min() - TDelta
    y2_max = df["Humidity"].max() + HDelta
    y2_min = df["Humidity"].min() - HDelta
    graph_title = "Temperature & Humidity"
    fname = "temp"

    plt.subplot(2, 1, 1)
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    # fig.set_size_inches(10, 6)
    fig.set_size_inches(40, 10)
    # fig.set_size_inches(20, 12)

    color_1 = "tab:red"
    color_2 = "tab:blue"

    ax1.plot(df.index, y1, marker="s", label=y1_name, color=color_1)
    ax2.plot(df.index, y2, marker="o", label=y2_name, color=color_2)

    ax1.set_xlabel("datetime")
    ax1.set_ylabel(f"{y1_name} (Red)", fontsize=14, color=color_1)
    ax2.set_ylabel(f"{y2_name} (Blue)", fontsize=14, color=color_2)

    ax1.tick_params(axis="y", labelcolor=color_1)
    ax2.tick_params(axis="y", labelcolor=color_2)

    ax1.set_ylim(y1_min, y1_max)
    ax2.set_ylim(y2_min, y2_max)

    fig.tight_layout()
    ax1.set_title(f"Room {graph_title} Level", fontsize=16)
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.grid()

    plt.subplot(2, 2, 1)
    # Y Data
    y1 = df["Voltage"]
    y2 = df["DustDensity"]
    y1_name = "Voltage"
    y2_name = "DustDensity"
    # Y axis Max & Min Value
    y1_max = df["Voltage"].max() + VDelta
    y1_min = df["Voltage"].min() - VDelta
    y2_max = df["DustDensity"].max() + DDelta
    y2_min = df["DustDensity"].min() - DDelta
    graph_title = "Fine-Dust"
    fname = "finedust"

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    # fig.set_size_inches(10, 6)
    fig.set_size_inches(40, 10)
    # fig.set_size_inches(20, 12)

    color_1 = "tab:red"
    color_2 = "tab:blue"

    ax1.plot(df.index, y1, marker="s", label=y1_name, color=color_1)
    ax2.plot(df.index, y2, marker="o", label=y2_name, color=color_2)

    ax1.set_xlabel("datetime")
    ax1.set_ylabel(f"{y1_name} (Red)", fontsize=14, color=color_1)
    ax2.set_ylabel(f"{y2_name} (Blue)", fontsize=14, color=color_2)

    ax1.tick_params(axis="y", labelcolor=color_1)
    ax2.tick_params(axis="y", labelcolor=color_2)

    ax1.set_ylim(y1_min, y1_max)
    ax2.set_ylim(y2_min, y2_max)

    fig.tight_layout()
    ax1.set_title(f"Room {graph_title} Level", fontsize=16)
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.grid()
    plt.show()
    # fig.savefig(f"room_{CURRTIME}_{fname}.png")
    fig.savefig(f"{CURRTIME}_{file_list[userInput]}_{fname}.png")


def DrawGraph():
    TDelta = 1  # Temperature
    HDelta = 1  # Humidity
    VDelta = 0.3  # Voltage
    DDelta = 3  # DustDensity

    # Y Data
    y1 = df["Temperature"]
    y2 = df["Humidity"]
    y3 = df["Voltage"]
    y4 = df["DustDensity"]

    y1_name = "Temperature"
    y2_name = "Humidity"
    y3_name = "Voltage"
    y4_name = "DustDensity"

    # Y axis Max & Min Value
    y1_max = y1.max() + TDelta
    y1_min = y1.min() - TDelta
    y2_max = y2.max() + HDelta
    y2_min = y2.min() - HDelta
    y3_max = y3.max() + VDelta
    y3_min = y3.min() - VDelta
    y4_max = y4.max() + DDelta
    y4_min = y4.min() - DDelta

    graph_title_1 = f'{file_list[userInput].replace(".txt", "")} Temperature & Humidity'
    graph_title_2 = f'{file_list[userInput].replace(".txt", "")} Fine-Dust'
    fname = "temp"

    fig, (ax1, ax3) = plt.subplots(2, 1, sharex=True)
    ax2 = ax1.twinx()
    ax4 = ax3.twinx()
    # fig.set_size_inches(10, 6)
    # fig.set_size_inches(20, 10)
    fig.set_size_inches(20, 14)

    color_1 = "tab:red"
    color_2 = "tab:blue"

    ax1.plot(df.index, y1, marker="s", label=y1_name, color=color_1)
    ax2.plot(df.index, y2, marker="o", label=y2_name, color=color_2)
    ax3.plot(df.index, y3, marker="s", label=y3_name, color=color_1)
    ax4.plot(df.index, y4, marker="o", label=y4_name, color=color_2)

    ax1.set_xlabel("datetime")
    ax3.set_xlabel("datetime")
    ax1.set_ylabel(f"{y1_name} (Red)", fontsize=14, color=color_1)
    ax2.set_ylabel(f"{y2_name} (Blue)", fontsize=14, color=color_2)
    ax3.set_ylabel(f"{y3_name} (Red)", fontsize=14, color=color_1)
    ax4.set_ylabel(f"{y4_name} (Blue)", fontsize=14, color=color_2)

    ax1.tick_params(axis="y", labelcolor=color_1)
    ax2.tick_params(axis="y", labelcolor=color_2)
    ax3.tick_params(axis="y", labelcolor=color_1)
    ax4.tick_params(axis="y", labelcolor=color_2)

    ax1.set_ylim(y1_min, y1_max)
    ax2.set_ylim(y2_min, y2_max)
    ax3.set_ylim(y3_min, y3_max)
    ax4.set_ylim(y4_min, y4_max)

    fig.tight_layout()
    ax1.set_title(f"Room {graph_title_1} Level", fontsize=20)
    ax3.set_title(f"Room {graph_title_2} Level", fontsize=20)

    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    ax3.legend(loc="upper left")
    ax4.legend(loc="upper right")

    ax1.grid()
    ax3.grid()

    fig.tight_layout()

    plt.show()

    # fig.savefig(f"{CURRTIME}_{file_list[userInput]}.png")
    fig.savefig(f"cvt_{file_list[userInput]}.png")

## OUTPUT


In [ ]:
# raise Exception("End Of File!")

In [ ]:
GetDataList()

for i in range(len(file_list)):
    userInput = i - 1
    FILE_PATH = f"{dir_master}{file_list[userInput]}"
    print(f"\nFile Path Set as >> {FILE_PATH}")
    FileOpenRead()
    CreateDataFrame()

    DisplayColSum()

    wrong_data = [
        CheckIssue("Temperature"),
        CheckIssue("Humidity"),
        CheckIssue("Voltage"),
        CheckIssue("DustDensity"),
    ]
    if any(x != 0 for x in wrong_data):
        DisplayColSum()

    DrawGraph()